Without doing any calculations, there are several TF functions for shaping the data
* shape: changes the dimensions of a set of data (e.g. a 2x6 array becomes a 2x2x3 array)
* tranpose: flips an array along any chosen axes
* indexing: returns a lower dimensional tensor contained within the specified index of the input tensor
* slice: returns a continuous subset within the specified range of the input tensor
* gather: returns a tensor formed from catenating selected indicies of the input tensor
* gather_nd: returns a tensor formed from catenating selected subtensors of the input tensor
* split
* stack and unstack

### Transpose
Takes an tensor of data and an array which defines the new order of the indexes <br/>
Examples
* 2D array: [0, 1] does no permutation
* 2D array: [1, 0] transposes the array
* 3D array: [0, 2, 1] keeps the outermost order, but tranposes the inner dimensions
* 3D array: [2, 1, 0] keeps the middle order, but tranposes the first and last dimensions

In [1]:
import tensorflow as tf
import numpy as np

with tf.Session() as sess:
    x_np = np.array([[[1,2,3], [4,5,6]], [[51,52,53], [54,55,56]]])

    x_tf = tf.Variable(x_np, dtype=tf.float32)
    sess.run(tf.global_variables_initializer())
    print("Original\n", sess.run(x_tf))

    # Flip the first two indexes, but keep the order of the innermost index
    transpose_tf = tf.transpose(x_tf, perm=[1, 0, 2])
    sess.run(tf.global_variables_initializer())
    print("Transposed\n", sess.run(transpose_tf))



Original
 [[[  1.   2.   3.]
  [  4.   5.   6.]]

 [[ 51.  52.  53.]
  [ 54.  55.  56.]]]
Transposed
 [[[  1.   2.   3.]
  [ 51.  52.  53.]]

 [[  4.   5.   6.]
  [ 54.  55.  56.]]]


### Indexing
Each time indexing is done, the dimension of the result is reduced by one

In [2]:
with tf.Session() as sess:
    x0_tf = x_tf[0]  # reduces the dimensionality by 1
    sess.run(tf.global_variables_initializer())
    print("Index only along the first axis (2D result)\n", sess.run(x0_tf))
    
    x01_tf = x_tf[0][1]  # reduces the dimensionality by 2
    sess.run(tf.global_variables_initializer())
    print("Index along the first two axes (1D result)\n", sess.run(x01_tf))

    x012_tf = x_tf[0][1][2] # reduces the dimensionality by 3
    sess.run(tf.global_variables_initializer())
    print("Index along all three axes (scalar result)\n", sess.run(x012_tf))

    x012_tf = x_tf[0, 1, 2] # different syntax which produces same result
    sess.run(tf.global_variables_initializer())
    print("Index along all three axes (scalar result)\n", sess.run(x012_tf))



Index only along the first axis (2D result)
 [[ 1.  2.  3.]
 [ 4.  5.  6.]]
Index along the first two axes (1D result)
 [ 4.  5.  6.]
Index along all three axes (scalar result)
 6.0
Index along all three axes (scalar result)
 6.0


### Slicing
Slicing does NOT reduce the dimensionality of the result

In [3]:
with tf.Session() as sess:
    x0full0_tf = x_tf[:,:,1:]  # notice the commas.  This is not the same as [:][:][1:]
    sess.run(tf.global_variables_initializer())
    print("Take a slice in the last axis (3D result)\n", sess.run(x0full0_tf))

    # slice takes an array of initial indexes and an array of lengths
    x0full0_tf = tf.slice(x_tf, [0, 0, 1], [2, 2, 2])  # this is the same as the previous example
    sess.run(tf.global_variables_initializer())
    print("Take a slice in the last axis (3D result)\n", sess.run(x0full0_tf))

    x01range_tf = x_tf[0][1][1:]
    sess.run(tf.global_variables_initializer())
    print("Index along the first two axes and take a slice of the last (1D result)\n", sess.run(x01range_tf))



Take a slice in the last axis (3D result)
 [[[  2.   3.]
  [  5.   6.]]

 [[ 52.  53.]
  [ 55.  56.]]]
Take a slice in the last axis (3D result)
 [[[  2.   3.]
  [  5.   6.]]

 [[ 52.  53.]
  [ 55.  56.]]]
Index along the first two axes and take a slice of the last (1D result)
 [ 5.  6.]


### Gather
* Creates a new tensor by catenating selected indices from the input tensor
* `gather` only does selection based on indexes within the first dimension of the input tensor
* Passing a 1-D list for the `indices`
 * `gather` selects a subset based on a list of indexes that is provided
 * the tensor that is returned has the same number of dimensions as the input tensor
* Passing a 2-D list for the `indices`
 * `gather` does the same as for the 1-D case, but it does it multiple times
 * the tensor that is returned has one more dimension than the input tensor
* If selection is required on a dimension other than the first dimension, then `transpose` may be needed before using `gather`
[gather official doc](https://www.tensorflow.org/api_docs/python/tf/gather)

In [4]:
import tensorflow as tf
import numpy as np
with tf.Session() as sess:
    x_np = np.array([[[1,2,3], [4,5,6]], [[21,22,23], [24,25,26]], [[41,42,43], [44,45,46]]])

    x_tf = tf.Variable(x_np, dtype=tf.float32)
    sess.run(tf.global_variables_initializer())
    print("Original\n", sess.run(x_tf))
    
    # data is 3D.  Result is 3D by gathering together the 0th and 2nd 2D slices
    print("3D result from gathering [0, 2]\n", sess.run(tf.gather(x_tf, [0, 2])))
    # data is 3D.  Result is 4D by gathering together the 0th and 2nd 2D slices twice
    print("4D result from gathering [0, 2] twice\n", sess.run(tf.gather(x_tf, [[0, 2], [0, 2]])))


Original
 [[[  1.   2.   3.]
  [  4.   5.   6.]]

 [[ 21.  22.  23.]
  [ 24.  25.  26.]]

 [[ 41.  42.  43.]
  [ 44.  45.  46.]]]
3D result from gathering [0, 2]
 [[[  1.   2.   3.]
  [  4.   5.   6.]]

 [[ 41.  42.  43.]
  [ 44.  45.  46.]]]
4D result from gathering [0, 2] twice
 [[[[  1.   2.   3.]
   [  4.   5.   6.]]

  [[ 41.  42.  43.]
   [ 44.  45.  46.]]]


 [[[  1.   2.   3.]
   [  4.   5.   6.]]

  [[ 41.  42.  43.]
   [ 44.  45.  46.]]]]


### Gather_nd
* Creates a new tensor by catenating selected sub-tensors from the input tensor
* `gather_nd` allows indexing to select a subset from any depth within the input tensor
* To contrast `gather_nd` from `gather`, indices of `[1, 2]`
 * for `gather` this means select from positions 1 and position 2 within the first dimension
 * for `gather_nd` this means to select from position 1 in the first dimension and from position 2 within the second dimension
* Similar to `gather`, `gather_nd` can take an input that is 2D which will create a result that is effectively an array formed by repeating the 1D case multiple times

In [5]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print("Original\n", sess.run(x_tf))

    # Result is 1D since we have indexed two levels within a 3D tensor
    print("1D result from indices [1, 1]\n", sess.run(tf.gather_nd(x_tf, [1, 1])))
    # Result is 2D by gathering together two 1D slices (each 1D slice comes from indexed two levels within a 3D tensor)
    print("2D result from indices [[1, 1], [2, 0]]\n", sess.run(tf.gather_nd(x_tf, [[1, 1], [2, 0]])))


Original
 [[[  1.   2.   3.]
  [  4.   5.   6.]]

 [[ 21.  22.  23.]
  [ 24.  25.  26.]]

 [[ 41.  42.  43.]
  [ 44.  45.  46.]]]
1D result from indices [1, 1]
 [ 24.  25.  26.]
2D result from indices [[1, 1], [2, 0]]
 [[ 24.  25.  26.]
 [ 41.  42.  43.]]


### split
* separates an input tensor into multiple tensors
* input tensor is split based on sizes within the specified axis




In [6]:
import tensorflow as tf
import numpy as np
with tf.Session() as sess:
    w_np = np.array([[1,2], [3,4], [5,6], [7,8], [9,10], [11,12], [13,14], [15,16]])

    w_tf = tf.Variable(w_np, dtype=tf.float32)
    sess.run(tf.global_variables_initializer())
    print("Original\n", sess.run(w_tf))

    #split into three tensors separated on axis=0
    training, validation, testing = tf.split(w_tf, [4, 2, 2], 0)
    print("Training\n",   sess.run(training))
    print("Testing\n",    sess.run(testing))
    print("Validation\n", sess.run(validation))
    

Original
 [[  1.   2.]
 [  3.   4.]
 [  5.   6.]
 [  7.   8.]
 [  9.  10.]
 [ 11.  12.]
 [ 13.  14.]
 [ 15.  16.]]
Training
 [[ 1.  2.]
 [ 3.  4.]
 [ 5.  6.]
 [ 7.  8.]]
Testing
 [[ 13.  14.]
 [ 15.  16.]]
Validation
 [[  9.  10.]
 [ 11.  12.]]


### stack and unstack
* `stack` catenates a list of tensors along the specified axis
* example for `stack` below is taken directly from [tensorflow documentation](https://www.tensorflow.org/api_docs/python/tf/stack)
* `unstack` performs the opposite which will split a tensor apart into a list of tensors

In [7]:
with tf.Session() as sess:
    x_tf = tf.Variable(np.array([1, 4]), dtype=tf.float32)
    y_tf = tf.Variable(np.array([2, 5]), dtype=tf.float32)
    z_tf = tf.Variable(np.array([3, 6]), dtype=tf.float32)

    # Pack along the default of the first dimension (axis=0)
    s0_tf = tf.stack([x_tf, y_tf, z_tf])
    # Pack along the second dimension
    s1_tf = tf.stack([x_tf, y_tf, z_tf], axis=1)
    
    sess.run(tf.global_variables_initializer())
    print("Stack axis=0\n", sess.run(s0_tf))
    print("Stack axis=1\n", sess.run(s1_tf))

    # Unpack to see if the original tensors can be recovered (return value is a list of tensors)
    r0_tf = tf.unstack(s0_tf, axis=0)
    r1_tf = tf.unstack(s1_tf, axis=1)
    print("Recovered x by unstacking axis=0\n", sess.run(r0_tf[0])) # evaluate the first tensor in the list
    print("Recovered y by unstacking axis=0\n", sess.run(r0_tf[1]))
    print("Recovered z by unstacking axis=0\n", sess.run(r0_tf[2]))
    print("Recovered x by unstacking axis=1\n", sess.run(r1_tf[0]))
    print("Recovered y by unstacking axis=1\n", sess.run(r1_tf[1]))
    print("Recovered z by unstacking axis=1\n", sess.run(r1_tf[2]))

    

Stack axis=0
 [[ 1.  4.]
 [ 2.  5.]
 [ 3.  6.]]
Stack axis=1
 [[ 1.  2.  3.]
 [ 4.  5.  6.]]
Recovered x by unstacking axis=0
 [ 1.  4.]
Recovered y by unstacking axis=0
 [ 2.  5.]
Recovered z by unstacking axis=0
 [ 3.  6.]
Recovered x by unstacking axis=1
 [ 1.  4.]
Recovered y by unstacking axis=1
 [ 2.  5.]
Recovered z by unstacking axis=1
 [ 3.  6.]
